In [5]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
df22 = pd.read_csv("./data/서울특별시_전월세가_2022.csv", encoding='cp949')
df23 = pd.read_csv("./data/서울특별시_전월세가_2023.csv", encoding='utf-8')

In [4]:
df22.columns

Index(['접수년도', '자치구코드', '자치구명', '법정동코드', '법정동명', '지번구분코드', '지번구분', '본번', '부번',
       '층', '계약일', '전월세구분', '임대면적', '보증금(만원)', '임대료(만원)', '건물명', '건축년도',
       '건물용도', '계약기간', '신규계약구분', '갱신청구권사용', '종전보증금', '종전임대료'],
      dtype='object')

### 데이터 추출
>   - 청년 안심 주택 데이터 추출

In [6]:
import requests
from bs4 import BeautifulSoup
url = "https://soco.seoul.go.kr/youth/pgm/home/yohome/maplist.json"
head = {"User-Agent":
"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"}


response = requests.post(url, headers= head)

ansim = response.json()
ansim_df = pd.DataFrame(ansim['mapResultList'])

>   - LH & SH 데이터 추출

In [8]:
lhlist = pd.read_csv("./data/한국토지주택공사_(매입임대)마이홈포털 기존임대주택 목록_20240104.csv", encoding='cp949')
lhlist_happy = pd.read_csv("./data/한국토지주택공사_(행복주택)마이홈포털 기존임대주택 목록_20240104.csv", encoding='cp949')
select_col = ['adresGu','adres', 'scaleTot','roomTotArea','moneyDepositLow',
       'fnMoneyDepositLow', 'fnMoneyDepositHigh', 'fnMoneyRentalLow',
       'fnMoneyRentalHigh']
ansim2 = ansim_df[select_col]
a = ansim2['roomTotArea'].tolist()

/tmp/ipykernel_14127/1769481165.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  lhlist = pd.read_csv("./data/한국토지주택공사_(매입임대)마이홈포털 기존임대주택 목록_20240104.csv", encoding='cp949')


### 데이터 전처리
>   - 보증금 및 월세 int 변환

In [9]:
import re

# 주어진 리스트
data_list = a

# 알파벳만을 제거하는 정규식 패턴
pattern = re.compile(r'[A-Za-z]+')
# 괄호와 괄호 안에 있는 내용을 지우는 정규식 패턴
pattern2 = re.compile(r'\(.*?\)')

result_list = [re.sub(pattern, '', value) for value in data_list]
# 리스트 컴프리헨션을 사용하여 각 값에서 괄호와 그 안에 있는 내용을 제거
result_list = [re.sub(pattern2, '', value) for value in result_list]

# 결과 출력
print(result_list)
print()
print()

['17.05', '17', '18.03', '17.02', '17', '17.78', '18', '16', '17', '14', '20', '17-2,3', '17', '16', '14', '17', '15', '17', '17', '20', '16', '16', '15.51', '16.3', '16', '-15.33', '16', '16.12', '20.08', '15', '15', '17', '14', '17', '14', '16.63', '19', '15.73', '15', '14', '14.35', '18', '17', '15', '17', '16', '16', '14', '17', '16']




In [10]:
result_list[result_list.index('17-2,3')] = 17
result_list[result_list.index('-15.33')] = 15.33
ansim2['roomTotArea'] = result_list
ansim2['roomTotArea'] = ansim2['roomTotArea'].astype(float)

/tmp/ipykernel_14127/2162907260.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ansim2['roomTotArea'] = result_list
/tmp/ipykernel_14127/2162907260.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ansim2['roomTotArea'] = ansim2['roomTotArea'].astype(float)


In [11]:
ansim2[ansim2['moneyDepositLow'].isnull()]

,adresGu,adres,scaleTot,roomTotArea,moneyDepositLow,fnMoneyDepositLow,fnMoneyDepositHigh,fnMoneyRentalLow,fnMoneyRentalHigh
14,중구,중구 마른내로 159-4,105,14.0,NaN,-6502,-6502,-6502,-6502


In [166]:
ansim2['fnMoneyRentalLow'][12] = '62000'

/tmp/ipykernel_12470/82292642.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ansim2['fnMoneyRentalLow'][12] = '62000'


In [168]:
# 리스트 컴프리헨션을 사용하여 각 문자열에서 'a'를 'x'로 바꾸기
ansim2['fnMoneyRentalLow'] = [''.join(['0000' if char == '만' else char for char in word]) for word in ansim2['fnMoneyRentalLow']]

/tmp/ipykernel_12470/230215164.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ansim2['fnMoneyRentalLow'] = [''.join(['0000' if char == '만' else char for char in word]) for word in ansim2['fnMoneyRentalLow']]


In [171]:
ansim2['fnMoneyRentalHigh'][4] = '1060000'
ansim2['fnMoneyRentalHigh'][7] = '1030000'
ansim2['fnMoneyRentalHigh'][12] = '476000'
ansim2['fnMoneyRentalHigh'][42] = '1120000'

/tmp/ipykernel_12470/995886632.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ansim2['fnMoneyRentalHigh'][4] = '1060000'
/tmp/ipykernel_12470/995886632.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ansim2['fnMoneyRentalHigh'][7] = '1030000'
/tmp/ipykernel_12470/995886632.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ansim2['fnMoneyRentalHigh'][12] = '476000'
/tmp/ipykernel_12470/995886632.py:4: SettingWithCopyWarning: 

In [172]:
# 리스트 컴프리헨션을 사용하여 각 문자열에서 'a'를 'x'로 바꾸기
ansim2['fnMoneyRentalHigh'] = [''.join(['0000' if char == '만' else char for char in word]) for word in ansim2['fnMoneyRentalHigh']]

/tmp/ipykernel_12470/1324027442.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ansim2['fnMoneyRentalHigh'] = [''.join(['0000' if char == '만' else char for char in word]) for word in ansim2['fnMoneyRentalHigh']]


In [190]:
# 주어진 리스트
a = ansim2['fnMoneyDepositLow'].tolist()
b = ansim2['fnMoneyDepositHigh'].tolist()

# '백0만'을 '백만'으로, '천0만'을 '천만'으로 바꾸는 함수
def replace_baek(word):
    return word.replace('백0만', '백만').replace('천0만', '천만')

# 각 문자열에서 '백0만'을 '백만'으로, '천0만'을 '천만'으로 바꾸기
new_list = [replace_baek(word) for word in a]
new_list2 = [replace_baek(word) for word in b]

# 결과 출력
print(new_list)

ansim2['fnMoneyDepositLow'] = new_list
ansim2['fnMoneyDepositHigh'] = new_list2

['4천만', '4천3백만', '3천6백만', '3천6백만', '4천5백만', '3천4백만', '4천2백만', '4천6백만', '3천8백만', '4천4백80만', '2천6백40만', '4천만', '3천7백13만', '3천만60만', '5천8백만', '5천4백만', '5천6백만', '4백40만', '3천8백25만', '4천4백만', '3천9백만', '4천6백만', '3천2백만', '2천7백79만', '3천8백만', '2천8백24만', '3천5백만', '5천만', '5천만34만', '2천4백만', '3백68만', '4천1백57만', '1천7백만', '1천1백만5만', '2천4백75만', '2천2백84만', '3천3백만4만', '1천8백63만', '1천9백29만', '1천4백96만', '2천9백만4만', '6천1백20만', '3천6백40만', '6천8백만', '4백90만', '4천4백만', '3천1백62만', '3천7백만', '3천7백만']


/tmp/ipykernel_12470/1979941606.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ansim2['fnMoneyDepositLow'] = new_list
/tmp/ipykernel_12470/1979941606.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ansim2['fnMoneyDepositHigh'] = new_list2


In [204]:
c = ansim2['fnMoneyDepositLow'].tolist()
d = ansim2['fnMoneyDepositHigh'].tolist()

In [206]:
import re

# 주어진 문자열 리스트
strings = ['4천만', '4천3백만', '3천6백만', '1억9천1백만']

# 정규식 패턴 정의
pattern = re.compile(r'(\d+)(천|백|만|억)?')

# 문자열을 숫자로 변환하는 함수
def convert_to_number(word):
    number = 0
    matches = pattern.findall(word)
    for match in matches:
        digit = int(match[0])
        unit = match[1]
        if unit == '천':
            digit *= 10000000
        elif unit == '백':
            digit *= 1000000
        elif unit == '만':
            digit *= 10000
        elif unit == '억':
            digit *= 100000000  # 억 단위는 1억 = 1e8
        else:
            digit *= 1
        number += digit
    return number

# 각 문자열을 숫자로 변환하여 리스트 생성
new_list = [convert_to_number(word) for word in c]
new_list2 = [convert_to_number(word) for word in d]

# 결과 출력
print(new_list)


[40000000, 43000000, 36000000, 36000000, 45000000, 34000000, 42000000, 46000000, 38000000, 44800000, 26400000, 40000000, 37130000, 30600000, 58000000, 54000000, 56000000, 4400000, 38250000, 44000000, 39000000, 46000000, 32000000, 27790000, 38000000, 28240000, 35000000, 50000000, 50340000, 24000000, 3680000, 41570000, 17000000, 11050000, 24750000, 22840000, 33040000, 18630000, 19290000, 14960000, 29040000, 61200000, 36400000, 68000000, 4900000, 44000000, 31620000, 37000000, 37000000]


In [207]:
ansim2['fnMoneyDepositLow'] = new_list
ansim2['fnMoneyDepositHigh'] = new_list2

/tmp/ipykernel_12470/3633027788.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ansim2['fnMoneyDepositLow'] = new_list
/tmp/ipykernel_12470/3633027788.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ansim2['fnMoneyDepositHigh'] = new_list2


In [214]:
ansim2[['moneyDepositLow', 'B']] = df[['A', 'B']].astype(int)

float64
